In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.metrics import mean_squared_error
import os
import sys
sys.path.append('../src/')
import util as util
from factorisedms import impute_by_matrix_factorisation
from aalasso import linear_collaborative_filtering_on_rows_lasso
from wu2019.wu_modify import foo_wu2019_wrapping_func_23031601

Duplicate key in file '/Users/fu.j/.matplotlib/matplotlibrc', line 2 ('backend: TkAgg')


In [2]:
DMS_ENVF = pd.read_csv('../data/dms_envfeat_data.csv', index_col=0)

# Benchmark AALasso, FactoriseDMS, Envision and Wu2019 methods

## Data preparation

In [3]:
def foo_make_featured_data_23031601(input_data):
    dms_envf_all = input_data.copy()
    dms_envf_all['wt_mut'] = dms_envf_all['aa1'] + dms_envf_all['aa2']  # NA was assumed as missing.
    env_feats = ['aa1', 'aa2', 'wt_mut', 'aa1_polarity', 'aa2_polarity', 'aa1_PI', 'aa2_PI', 
                 'deltaPI', 'aa1_weight', 'aa2_weight', 'deltaWeight', 'aa1vol', 'aa2vol', 
                 'deltavolume', 'Grantham', 'aa1_psic', 'aa2_psic', 'delta_psic', 'accessibility', 
                 'dssp_sec_str', 'phi_psi_reg', 'delta_solvent_accessibility', 'b_factor', 
                 'mut_msa_congruency', 'mut_mut_msa_congruency', 'seq_ind_closest_mut', 
                 'evolutionary_coupling_avg']
    categ_feat = ['aa1', 'aa2', 'wt_mut', 'aa1_polarity', 'aa2_polarity', 'dssp_sec_str', 'phi_psi_reg']
    numer_feat = [x for x in env_feats if x not in categ_feat]
    dms_envf_all = util.impute_missing_value(dms_envf_all, categ_feat, numer_feat)
    dms_envf_all, encoded_col = util.encode_categorical_feature(dms_envf_all, categ_feat, ['aa1', 'aa2'])
    env_encfeat = encoded_col + numer_feat

    wu_feature = []
    blosum = pd.read_csv('../src/wu2019/database/humandb/dms/other_features/blosums.csv')
    blosum = blosum[['aa_ref', 'aa_alt', 'blosum100']]
    folder = '../src/wu2019/database/humandb/dms/features/'
    for unip in dms_envf_all.uniprot_id.unique():
        file = f"{folder}{unip}_features.csv"
        if os.path.isfile(file):
            data = pd.read_csv(file)
            data['uniprot_id'] = unip
            wu_feature.append(data)
    wu_feature = pd.concat(wu_feature, ignore_index=True)
    wu_feature = pd.merge(wu_feature, blosum, on=['aa_ref', 'aa_alt'], how='left', validate='m:1')
    wu_feature = wu_feature[['polyphen_score', 'provean_score', 'blosum100', 'aa_alt', 'aa_ref', 'aa_pos', 'uniprot_id']]
    wu_feature = wu_feature.rename(columns={'aa_alt': 'aa2', 'aa_ref': 'aa1', 'aa_pos': 'u_pos'})
    
    dms_envwu_all = pd.merge(dms_envf_all, wu_feature, on=['uniprot_id', 'u_pos', 'aa1', 'aa2'], how='left', 
                             validate='m:1')
    dms_envwu_all = util.impute_missing_value(dms_envwu_all, None, ['polyphen_score', 'provean_score', 'blosum100'])
    
    return dms_envwu_all, env_encfeat

In [4]:
basic_ens_feat = ['envision_pred', 'lasso_pred', 'fdms_pred', 'row-mean_pred', 'wu2019_pred', 
                  'polyphen_score', 'provean_score', 'blosum100']  # Seems that Wu2019 only use these features?
ens_params = {'n_estimators': [100, 200, 500], 'max_depth': [5, 10, 50], 'min_weight_fraction_leaf': [0, 0.01, 0.1]}
env_params = {'max_features': [0.1, 0.5, None], 'max_depth': [3, 7, 15], 'min_weight_fraction_leaf': [0, 0.001, 0.01]}

dms_envwu_all, env_encfeat = foo_make_featured_data_23031601(DMS_ENVF)
wu_ava_unip = pd.read_csv('../src/wu2019/projects/imputation/gwt/www/downloads/supported_uniprot_ids.txt',
                  sep='\t')
wu_ava_unip = set(wu_ava_unip.UniProtID)

## Modelling

In [ ]:
output_dir = '../result/benchmark_ensemble/'
aalasso_dir = '../result/aalasso/'

sess_prefix = 'env_tune'
for dms_id in dms_envwu_all.dms_id.unique():
    seed = 0
    np.random.seed(seed)
    dms_data = dms_envwu_all.query("dms_id == @dms_id")
    for rep in range(5):
        tr_data, te_data = train_test_split(dms_data, test_size=0.1, random_state=seed+rep, shuffle=True)

        cv_data = {'full': {'tr': tr_data, 'te': te_data}}
        for cv_key, (train_cv, valid_cv) in enumerate(KFold(10, random_state=seed+rep, shuffle=True).split(tr_data)):
            cv_data[cv_key] = {'tr': tr_data.iloc[train_cv], 'te': tr_data.iloc[valid_cv]}

        for cv_key, data in cv_data.items():
            train_cv = data['tr']
            valid_cv = data['te']
            valid_cv['cv'] = cv_key

            # Add residues only available in the validation data. This is necessary for certain models when the data 
            # is exremely sparse.
            va_only_pos = set(valid_cv.position) - set(train_cv.position)
            if len(va_only_pos) > 0:
                va_only_wt = valid_cv.query("position in @va_only_pos")[['dms_id', 'position', 'u_pos',
                                                                         'aa1']].drop_duplicates().copy()
                va_only_wt['aa2'] = va_only_wt['aa1']
                va_only_wt['score'] = 1
                train_cv_va_wt = pd.concat([train_cv, va_only_wt], ignore_index=True)
                mask_mat = util.create_dms_score_matrix(train_cv_va_wt, 1)
            else:
                train_cv_va_wt = train_cv.copy()
                mask_mat = util.create_dms_score_matrix(train_cv, 1)

            # Individual imputation methods.
            for method in ['envision', 'wu2019', 'lasso', 'fdms', 'row-mean']:
                if method == 'envision':
                    estimator = GradientBoostingRegressor(random_state=seed+rep)
                    gs = GridSearchCV(estimator, env_params)
                    estimator = gs.fit(train_cv[env_encfeat], train_cv['score']).best_estimator_
                    valid_cv['envision_pred'] = estimator.predict(valid_cv[env_encfeat])
                    if cv_key == 'full':
                        top_feat = list(pd.Series(dict(zip(env_encfeat, 
                                                           estimator.feature_importances_))).sort_values().tail(10).index)
                        ens_feat = basic_ens_feat + top_feat

                elif method == 'wu2019':
                    uniprot_id = train_cv['uniprot_id'].iloc[0]
                    if uniprot_id not in wu_ava_unip:
                        valid_cv['wu2019_pred'] = 1  # Can not be imputed so just set wildtype arbitrarily.
                    else:
                        imp_result = foo_wu2019_wrapping_func_23031601(train_cv_va_wt, f"{sess_prefix}_{dms_id}_{rep}_{cv_key}")
                        imp_result = imp_result.rename(columns={'pred_score': 'wu2019_pred'})
                        valid_cv = pd.merge(valid_cv, imp_result, on=['u_pos', 'aa2'], how='left', validate='1:1')

                else:
                    if method == 'lasso':
                        if cv_key == 'full':
                            coef_file = f"{aalasso_dir}{dms_id}_{rep}_coef.csv"
                        else:
                            coef_file = None
                        imp_mat = linear_collaborative_filtering_on_rows_lasso(mask_mat, 'ACDEFGHIKLMNPQRSTVWY', 
                                                                               0.01, seed+rep, coef_file)
                    elif method == 'fdms':
                        imp_mat = impute_by_matrix_factorisation(mask_mat, seed+rep, 100, n_components=2, 
                                                                           regularization=1)
                    elif method == 'row-mean':
                        null_mat = mask_mat.copy()
                        null_mat.loc[:,:] = np.nan
                        imp_mat = null_mat.T.fillna(mask_mat.mean(axis=1)).T
                    else:
                        raise ValueError()
                    imp_long = imp_mat.reset_index().melt(id_vars=['dms_id', 'position'], value_name=f"{method}_pred", 
                                                          var_name='aa2')
                    valid_cv = pd.merge(valid_cv, imp_long, on=['dms_id', 'position', 'aa2'], how='left', validate='1:1')

            cv_data[cv_key]['te'] = valid_cv
        # Ensemble method.
        tr_pred = pd.concat([cv_data[i]['te'] for i in range(10)], ignore_index=True)
        te_pred = cv_data['full']['te']
        estimator = RandomForestRegressor(random_state=seed+rep, n_jobs=2)
        gs = GridSearchCV(estimator, ens_params)
        estimator = gs.fit(tr_pred[ens_feat], tr_pred['score']).best_estimator_
        tr_pred['pred_score'] = estimator.predict(tr_pred[ens_feat])
        te_pred['pred_score'] = estimator.predict(te_pred[ens_feat])

        tr_pred.to_csv(f"{output_dir}{dms_id}_{rep}_enstrain_data.csv")
        te_pred.to_csv(f"{output_dir}{dms_id}_{rep}_enstest_data.csv")
        tuned_hp = pd.Series(estimator.get_params()).loc[list(ens_params.keys())]
        tuned_hp.to_csv(f"{output_dir}{dms_id}_{rep}_enstrain_hp.csv")
        imp = pd.Series(dict(zip(ens_feat, estimator.feature_importances_)))
        imp.to_csv(f"{output_dir}{dms_id}_{rep}_enstrain_imp.csv")

## Measure benchmark performance

In [12]:
# Envision features are not available for these proteins according to their website.
no_envision = ['A0A3G4RHW3', 'Q45996', 'P02829', 'P02943', 'P06654', 
               'P0ABQ4', 'P0DP23', 'P31016', 'P40515', 'P42212', 'P62593',
               'Q06851', 'P62554', 'Q14524']
no_env_dms = dms_envwu_all.query("uniprot_id in @no_envision")['dms_id'].unique()

envtune_dir = '../result/benchmark_ensemble/'
bench_perf = []
for file in os.listdir(envtune_dir):
    if file[-16:] != 'enstest_data.csv':
        continue
    
    enstest = pd.read_csv(f"{envtune_dir}{file}", index_col=0)
    dms_id, rep, _, _ = file.split('_')
    
    eval_dict = {'dms_id': dms_id, 'repeat': rep}
    for method in ['envision', 'wu2019', 'lasso', 'fdms', 'row-mean', 'ensemble']:
        if method == 'ensemble':
            pred_col = 'pred_score'
        else:
            pred_col = f"{method}_pred"
            
        if (method == 'envision') & (dms_id in no_env_dms):
            eval_dict[f"{method}_spear"] = eval_dict[f"{method}_pears"] = eval_dict[f"{method}_rmse"] = np.nan
        else:
            eval_dict[f"{method}_spear"] = spearmanr(enstest['score'], enstest[pred_col])[0]
            eval_dict[f"{method}_pears"] = pearsonr(enstest['score'], enstest[pred_col])[0]
            eval_dict[f"{method}_rmse"] = np.sqrt(mean_squared_error(enstest['score'], enstest[pred_col]))
    bench_perf.append(eval_dict)
bench_perf = pd.DataFrame(bench_perf)
#bench_perf.to_csv(f'{envtune_dir}performance.csv')

## Compute the absolute mean of AALasso coefficient values

In [11]:
root = '../result/aalasso/'
coef_sum = None
count = 0
for file_name in os.listdir(root):
    if file_name[-8:] != 'coef.csv':
        continue
    dms, rep, _ = file_name.split('_')
    if coef_sum is None:
        coef_sum = pd.read_csv(f"{root}{file_name}", index_col=0).abs()
    else:
        coef_sum = coef_sum + pd.read_csv(f"{root}{file_name}", index_col=0).abs()
    count += 1
coef_sum = (coef_sum / count).copy()
#coef_sum.to_csv(f'{root}coef_sum.csv')

# Performance with different trianing data proportion

## Data preparation

In [6]:
dms_envf = DMS_ENVF.copy()

# Envision features are not available for these proteins according to their website.
no_envision = ['A0A3G4RHW3', 'Q45996', 'P02829', 'P02943', 'P06654', 
               'P0ABQ4', 'P0DP23', 'P31016', 'P40515', 'P42212', 'P62593',
               'Q06851', 'P62554', 'Q14524']
dms_envf = dms_envf.query("uniprot_id not in @no_envision")
dms_envf['wt_mut'] = dms_envf['aa1'] + dms_envf['aa2']  # NA was assumed as missing.

env_feats = ['aa1', 'aa2', 'wt_mut', 'aa1_polarity', 'aa2_polarity', 'aa1_PI', 'aa2_PI', 
             'deltaPI', 'aa1_weight', 'aa2_weight', 'deltaWeight', 'aa1vol', 'aa2vol', 
             'deltavolume', 'Grantham', 'aa1_psic', 'aa2_psic', 'delta_psic', 'accessibility', 
             'dssp_sec_str', 'phi_psi_reg', 'delta_solvent_accessibility', 'b_factor', 
             'mut_msa_congruency', 'mut_mut_msa_congruency', 'seq_ind_closest_mut', 
             'evolutionary_coupling_avg']
categ_feat = ['aa1', 'aa2', 'wt_mut', 'aa1_polarity', 'aa2_polarity', 'dssp_sec_str', 'phi_psi_reg']
numer_feat = [x for x in env_feats if x not in categ_feat]

dms_envf = util.impute_missing_value(dms_envf, categ_feat, numer_feat)
dms_envf, encoded_col = util.encode_categorical_feature(dms_envf, categ_feat, ['aa1', 'aa2'])
env_encfeat = encoded_col + numer_feat

dms_complete = dict()
for dms, data in dms_envf.groupby('dms_id'):
    dms_complete[dms] = len(data) / (len(data.u_pos.unique()) * 19)
pick_dms = pd.Series(dms_complete)
pick_dms = pick_dms[pick_dms>=(18/19)]
pick_dms = list(pick_dms.index)
pick_dms_data = dms_envf.query("dms_id in @pick_dms").copy()
wu_ava_unip = pd.read_csv('../src/wu2019/projects/imputation/gwt/www/downloads/supported_uniprot_ids.txt',
                  sep='\t')
wu_ava_unip = set(wu_ava_unip.UniProtID)
pick_dms_data = pick_dms_data.query("uniprot_id in @wu_ava_unip")

wu_feature = []
blosum = pd.read_csv('../src/wu2019/database/humandb/dms/other_features/blosums.csv')
blosum = blosum[['aa_ref', 'aa_alt', 'blosum100']]
folder = '../src/wu2019/database/humandb/dms/features/'
for unip in dms_envf.uniprot_id.unique():
    file = f"{folder}{unip}_features.csv"
    if os.path.isfile(file):
        data = pd.read_csv(file)
        data['uniprot_id'] = unip
        wu_feature.append(data)
wu_feature = pd.concat(wu_feature, ignore_index=True)
wu_feature = pd.merge(wu_feature, blosum, on=['aa_ref', 'aa_alt'], how='left', validate='m:1')
wu_feature = wu_feature[['polyphen_score', 'provean_score', 'blosum100', 'aa_alt', 'aa_ref', 'aa_pos', 'uniprot_id']]
wu_feature = wu_feature.rename(columns={'aa_alt': 'aa2', 'aa_ref': 'aa1', 'aa_pos': 'u_pos'})

pick_dms_data = pd.merge(pick_dms_data, wu_feature, on=['uniprot_id', 'u_pos', 'aa1', 'aa2'], how='left', 
                         validate='m:1')
pick_dms_data = util.impute_missing_value(pick_dms_data, None, ['polyphen_score', 'provean_score', 'blosum100'])

## Modelling

In [7]:
te_ratio_vs_full = 0.1

sess_prefix = 'env_tune'
basic_ens_feat = ['envision_pred', 'lasso_pred', 'fdms_pred', 'row-mean_pred', 'wu2019_pred', 
                  'polyphen_score', 'provean_score', 'blosum100']  # Seems that Wu2019 only use these features?
ens_params = {'n_estimators': [100, 200, 500], 'max_depth': [5, 10, 50], 'min_weight_fraction_leaf': [0, 0.01, 0.1]}
env_params = {'max_features': [0.1, 0.5, None], 'max_depth': [3, 7, 15], 'min_weight_fraction_leaf': [0, 0.001, 0.01]}
output_dir = '../result/train_proportion/'

In [ ]:
for work_dms in pick_dms_data.dms_id.unique():
    for tr_ratio_vs_full in np.arange(0.1,1,0.1):

        seed = 0
        np.random.seed(seed)
        completeness_perf = []
        dms = work_dms
        dms_data = pick_dms_data.query("dms_id==@dms")
        completeness = dms_complete[dms]
        for rep in range(5):
            test_size = te_ratio_vs_full
            tr_data_all, te_data = train_test_split(dms_data, test_size=test_size, random_state=seed+rep, shuffle=True)

            if tr_ratio_vs_full == 0.9:  # Use all for training but test_size<=0 is not allowed
                tr_data = tr_data_all.copy()
            else:
                discard_size = 1 - (tr_ratio_vs_full / (1 - te_ratio_vs_full))
                tr_data, _ = train_test_split(tr_data_all, test_size=discard_size, random_state=seed+rep+1, 
                                              shuffle=True)

            cv_data = {'full': {'tr': tr_data, 'te': te_data}}
            for cv_key, (train_cv, valid_cv) in enumerate(KFold(10, random_state=seed+rep, shuffle=True).split(tr_data)):
                cv_data[cv_key] = {'tr': tr_data.iloc[train_cv], 'te': tr_data.iloc[valid_cv]}

            for cv_key, data in cv_data.items():
                train_cv = data['tr']
                valid_cv = data['te']
                valid_cv['cv'] = cv_key

                # Add residues only available in the validation data. This is necessary for certain models when the data 
                # is exremely sparse.
                va_only_pos = set(valid_cv.position) - set(train_cv.position)
                if len(va_only_pos) > 0:
                    va_only_wt = valid_cv.query("position in @va_only_pos")[['dms_id', 'position', 'u_pos',
                                                                             'aa1']].drop_duplicates().copy()
                    va_only_wt['aa2'] = va_only_wt['aa1']
                    va_only_wt['score'] = 1
                    train_cv_va_wt = pd.concat([train_cv, va_only_wt], ignore_index=True)
                    mask_mat = util.create_dms_score_matrix(train_cv_va_wt, 1)
                else:
                    train_cv_va_wt = train_cv.copy()
                    mask_mat = util.create_dms_score_matrix(train_cv, 1)

                # Individual imputation methods.
                for method in ['envision', 'wu2019', 'lasso', 'fdms', 'row-mean']:
                    if method == 'envision':
                        estimator = GradientBoostingRegressor(random_state=seed+rep)
                        gs = GridSearchCV(estimator, env_params)
                        estimator = gs.fit(train_cv[env_encfeat], train_cv['score']).best_estimator_
                        valid_cv['envision_pred'] = estimator.predict(valid_cv[env_encfeat])
                        if cv_key == 'full':
                            top_feat = list(pd.Series(dict(zip(env_encfeat, 
                                                               estimator.feature_importances_))).sort_values().tail(10).index)
                            ens_feat = basic_ens_feat + top_feat

                    elif method == 'wu2019':
                        uniprot_id = train_cv['uniprot_id'].iloc[0]
                        if uniprot_id not in wu_ava_unip:
                            valid_cv['wu2019_pred'] = 1  # Can not be imputed so just set wildtype arbitrarily.
                        else:
                            imp_result = foo_wu2019_wrapping_func_23031601(train_cv_va_wt, f"{sess_prefix}_{dms}_{rep}_{cv_key}")
                            imp_result = imp_result.rename(columns={'pred_score': 'wu2019_pred'})
                            valid_cv = pd.merge(valid_cv, imp_result, on=['u_pos', 'aa2'], how='left', validate='1:1')

                    else:
                        if method == 'lasso':
                            imp_mat = linear_collaborative_filtering_on_rows_lasso(mask_mat, 'ACDEFGHIKLMNPQRSTVWY', 
                                                                                   0.01, seed+rep)
                        elif method == 'fdms':
                            imp_mat = impute_by_matrix_factorisation(mask_mat, seed+rep, 100, n_components=2, 
                                                                               regularization=1)
                        elif method == 'row-mean':
                            null_mat = mask_mat.copy()
                            null_mat.loc[:,:] = np.nan
                            imp_mat = null_mat.T.fillna(mask_mat.mean(axis=1)).T
                        else:
                            raise ValueError()
                        imp_long = imp_mat.reset_index().melt(id_vars=['dms_id', 'position'], value_name=f"{method}_pred", 
                                                              var_name='aa2')
                        valid_cv = pd.merge(valid_cv, imp_long, on=['dms_id', 'position', 'aa2'], how='left', validate='1:1')

                cv_data[cv_key]['te'] = valid_cv

            # Ensemble method.
            tr_pred = pd.concat([cv_data[i]['te'] for i in range(10)], ignore_index=True)
            te_pred = cv_data['full']['te']
            estimator = RandomForestRegressor(random_state=seed+rep, n_jobs=2)
            gs = GridSearchCV(estimator, ens_params)
            estimator = gs.fit(tr_pred[ens_feat], tr_pred['score']).best_estimator_
            tr_pred['pred_score'] = estimator.predict(tr_pred[ens_feat])
            te_pred['pred_score'] = estimator.predict(te_pred[ens_feat])

            te_pred.to_csv(f"{output_dir}{tr_ratio_vs_full}_{dms}_{rep}_enstest_data.csv")
            tuned_hp = pd.Series(estimator.get_params()).loc[list(ens_params.keys())]
            tuned_hp.to_csv(f"{output_dir}{tr_ratio_vs_full}_{dms}_{rep}_enstrain_hp.csv")
            imp = pd.Series(dict(zip(ens_feat, estimator.feature_importances_)))
            imp.to_csv(f"{output_dir}{tr_ratio_vs_full}_{dms}_{rep}_enstrain_imp.csv")

### Measure performance

In [13]:
wkdir = '../result/train_proportion/'
ens_comp_perf = []
for file in os.listdir(wkdir):
    if file[-len('enstest_data.csv'):] != 'enstest_data.csv':
        continue
    compl, dms, rep, _, _ = file.split('_')
    data = pd.read_csv(f"{wkdir}{file}", index_col=0)
    row = {'dms_id': dms, 'repeat': rep, 'completeness': compl}
    for col in ['envision_pred', 'wu2019_pred', 'lasso_pred', 'fdms_pred', 'row-mean_pred', 'pred_score']:
        row[col] = pearsonr(data[col], data['score'])[0]
    ens_comp_perf.append(row)
ens_comp_perf = pd.DataFrame(ens_comp_perf)
#ens_comp_perf.to_csv('../result/train_proportion/performance.csv')